<a href="https://colab.research.google.com/github/shubham-ai/ChatGPT/blob/main/Sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install "langchain>=0.0.218" openai -qqq
!pip install "langchain"  --user

# assert langchain.__version__ >= "0.0.218", "Please ensure you are using LangChain v0.0.188 or higher"
import os
!pip install transformers[torch]
!pip install sentencepiece --user
!pip install einops --user
os.environ["Langchain"] = "langchain-testing"
!pip install vaderSentiment --user

In [4]:

!git lfs install
!git clone https://huggingface.co/google/flan-t5-base
# !git clone https://huggingface.co/google/flan-t5-xxl


Git LFS initialized.
Cloning into 'flan-t5-base'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 58 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (58/58), 621.24 KiB | 2.38 MiB/s, done.
Filtering content: 100% (5/5), 3.87 GiB | 48.88 MiB/s, done.


In [19]:
import langchain
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
import datetime
import os
import torch
# turn on wandb logging for langchain
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained("flan-t5-base")

# model = T5ForConditionalGeneration.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto", torch_dtype=torch.float16).to(device)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=512
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# llm = OpenAI(model_name="text-davinci-003", openai_api_key=API_KEY)

# first step in chain
start_time_ms = datetime.datetime.now().timestamp() * 1000

template='''You analyse the sentiment of the text provided.
The sentiment can be positive, negative or neutral.
{text}
'''
prompt_1 = PromptTemplate(

input_variables=["text"],

template=template)

chain_1 = LLMChain(llm = local_llm, prompt = prompt_1, verbose=False,output_key="sentiment")

# second step in chain

prompt_2 = PromptTemplate(

input_variables=["text"],

template='''tell me the percentage of politeness in the text provided. {text} ''')

chain_2 = LLMChain(llm = local_llm, prompt = prompt_2, verbose=False,output_key="politeness score")

# Third step in chain

prompt_3 = PromptTemplate(

input_variables=["text"],

template='''tell me only negative words found in the text provide. {text} ''')

chain_3 = LLMChain(llm = local_llm, prompt = prompt_3, verbose=False,output_key="negative words")
# fourth step in chain

prompt_4 = PromptTemplate(

input_variables=["text"],

template='''tell me only positive words found in the text provide. {text}''')

chain_4 = LLMChain(llm = local_llm, prompt = prompt_4, verbose=False,output_key="positive words")
overall_chain = SequentialChain(
    chains=[chain_1,chain_2,chain_3,chain_4],
    input_variables=["text"],
    # Here we return multiple variables
    output_variables=["sentiment","politeness score","negative words","positive words"],
    verbose=True)


end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
# text=" The battery is long lasting"
text=["go to hell","the battery is long lasting","hey how are you doing"]
for sentence in text:

  print(overall_chain({"text":sentence}))






> Entering new SequentialChain chain...

> Finished chain.
{'text': 'go to hell', 'sentiment': 'negative', 'politeness score': '0', 'negative words': 'go to hell', 'positive words': 'go to hell'}


> Entering new SequentialChain chain...

> Finished chain.
{'text': 'the battery is long lasting', 'sentiment': 'positive', 'politeness score': '14%', 'negative words': 'battery', 'positive words': 'battery'}


> Entering new SequentialChain chain...

> Finished chain.
{'text': 'hey how are you doing', 'sentiment': 'positive', 'politeness score': '80%', 'negative words': 'how are you doing', 'positive words': 'hey'}


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.5 MB/s eta 0:00:00


In [17]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

	# Create a SentimentIntensityAnalyzer object.
	sid_obj = SentimentIntensityAnalyzer()

	# polarity_scores method of SentimentIntensityAnalyzer
	# object gives a sentiment dictionary.
	# which contains pos, neg, neu, and compound scores.
	sentiment_dict = sid_obj.polarity_scores(sentence)

	print("Overall sentiment dictionary is : ", sentiment_dict)
	print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
	print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
	print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

	print("Sentence Overall Rated As", end = " ")

	# decide sentiment as positive, negative and neutral
	if sentiment_dict['compound'] >= 0.05 :
		print("Positive")

	elif sentiment_dict['compound'] <= - 0.05 :
		print("Negative")

	else :
		print("Neutral")



# Driver code
if __name__ == "__main__" :
  text=["go to hell","the battery is long lasting","hey how are you doing"]
  for i in text:
    print(i)
    sentiment_scores(i)



go to hell
Overall sentiment dictionary is :  {'neg': 0.697, 'neu': 0.303, 'pos': 0.0, 'compound': -0.6808}
sentence was rated as  69.69999999999999 % Negative
sentence was rated as  30.3 % Neutral
sentence was rated as  0.0 % Positive
Sentence Overall Rated As Negative
the battery is long lasting
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
sentence was rated as  0.0 % Negative
sentence was rated as  100.0 % Neutral
sentence was rated as  0.0 % Positive
Sentence Overall Rated As Neutral
hey how are you doing
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
sentence was rated as  0.0 % Negative
sentence was rated as  100.0 % Neutral
sentence was rated as  0.0 % Positive
Sentence Overall Rated As Neutral


In [12]:
text

'hey how are you doing'

In [ ]:
from langchain.llms.base import LLM
import torch

class CustomLLM(LLM):
    model_name = "upstage/llama-65b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        load_in_8bit=True,
        rope_scaling={"type": "dynamic", "factor": 2}  # allows handling of longer inputs
    )

    def _call(self, prompt, stop=None, **kwargs):
        prompt = f"### User:\n{prompt}\n\n### Assistant:\n"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        output = self.model.generate(**inputs, streamer=self.streamer, use_cache=True, max_new_tokens=float('inf'))
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=CustomLLM())